# Variational Auto Encoder based off of seq2seq for News Headline Generation
Created by Paul K. Mandal

This is a Variational Auto Encoder that will be used as a generator in a hybrid VAE-GAN architecture to generate news headlines. This would have not been possible without NicGan's VAE code which can be found here: https://github.com/NicGian/text_VAE/blob/master/text_VAE_v18.ipynb

In [2]:
import json

data = []
for line in open('Sarcasm_Headlines_Dataset.json', 'r'):
    data.append(json.loads(line))

In [3]:
titles = []
y_vals = []

for i in range(0,len(data)):
    titles.append(data[i]['headline'])
    y_vals.append(data[i]['is_sarcastic'])
    

In [4]:
#Let's now do some preprocessing

import nltk
nltk.download('punkt')
from nltk import word_tokenize

titles_tokenized = []
for title in titles:
    titles_tokenized.append(word_tokenize(title))

[nltk_data] Downloading package punkt to /home/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
titles_an = [] #alphanumeric
for title in titles_tokenized:
    words = [word for word in title if word.isalpha()]
    titles_an.append(words)

In [6]:
titles_an[0]

['former',
 'versace',
 'store',
 'clerk',
 'sues',
 'over',
 'secret',
 'code',
 'for',
 'minority',
 'shoppers']

In [7]:
#Let's now stem the words
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
titles_preprocessed = []
for title in titles_an:
    stemmed = [porter.stem(word) for word in title]
    titles_preprocessed.append(stemmed)

In [8]:
titles_preprocessed[0]

['former',
 'versac',
 'store',
 'clerk',
 'sue',
 'over',
 'secret',
 'code',
 'for',
 'minor',
 'shopper']

In [9]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

MAX_SEQUENCE_LENGTH = 20
MAX_NB_WORDS = 9999

#code from https://github.com/NicGian/text_VAE/blob/master/text_VAE_v18.ipynb
tokenizer = Tokenizer(MAX_NB_WORDS+1, oov_token='unk') #+1 for 'unk' token
tokenizer.fit_on_texts(titles_preprocessed)
print('Found %s unique tokens' % len(tokenizer.word_index))
## **Key Step** to make it work correctly otherwise drops OOV tokens anyway!
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= MAX_NB_WORDS} # <= because tokenizer is 1 indexed
tokenizer.word_index[tokenizer.oov_token] = MAX_NB_WORDS + 1
sequences = tokenizer.texts_to_sequences(titles_preprocessed)

Found 16437 unique tokens


In [10]:
sequences[0]

[371, 9792, 716, 2739, 1285, 56, 322, 1893, 6, 1286, 3309]

In [11]:
#Convert back to text
text = tokenizer.sequences_to_texts(sequences)
text[0]

'former versac store clerk sue over secret code for minor shopper'

In [12]:
word_index = tokenizer.word_index #the dict values start from 1 so this is fine with zeropadding
index2word = {v: k for k, v in word_index.items()}
print('Found %s unique tokens.' %len(word_index))

Found 9999 unique tokens.


In [13]:
x_vals = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [14]:
print('Shape of data tensor:', x_vals.shape)
NB_WORDS = (min(tokenizer.num_words, len(word_index))+1) #+1 for zero padding 

Shape of data tensor: (26709, 20)


In [15]:
x_real_vals = []
x_fake_vals = []

for i in range(0,len(x_vals)):
    if y_vals[i] == 0:
        x_real_vals.append(x_vals[i])
    else:
        x_fake_vals.append(x_vals[i])

In [16]:
x_real_vals = np.array(x_real_vals)
x_fake_vals = np.array(x_fake_vals)

In [17]:
x_vals.shape

(26709, 20)

In [18]:
x_real_vals.shape

(14985, 20)

In [19]:
type(x_real_vals)

numpy.ndarray

In [20]:
len(x_real_vals)

14985

In [21]:
x_test = x_real_vals[:1500]
x_train = x_real_vals[1500:]
x_val = x_train[:1500]
x_partial_train = x_train[1500:]

In [22]:
#These two lines need to be uncommented to download the weight embeddings if it has not been done before
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [23]:
EMBEDDING_DIM = 100

glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < NB_WORDS: #+1 for 'unk' oov token
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            glove_embedding_matrix[i] = embedding_vector
        else:
            # words not found in embedding index will the word embedding of unk
            glove_embedding_matrix[i] = embeddings_index.get('unk')
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))

Null word embeddings: 2


In [37]:
#https://github.com/twairball/keras_lstm_vae/
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
#from keras.preprocessing.sequence import pad_sequences
from keras.layers import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.models import Model
from scipy import spatial
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import csv
import os

#from tensorflow.python.framework.ops import disable_eager_execution
#isable_eager_execution()
mse = tf.keras.losses.MeanSquaredError()
max_len = 20
batch_size = 100
emb_dim = EMBEDDING_DIM
latent_dim = 64
intermediate_dim = 32
epsilon_std = 1.0
kl_weight = 0.01
num_sampled=500
act = ELU()

def create_lstm_vae(input_dim, 
    timesteps, 
    batch_size, 
    intermediate_dim, 
    latent_dim,
    epsilon_std=1.):

    """
    Creates an LSTM Variational Autoencoder (VAE). Returns VAE, Encoder, Generator. 
    # Arguments
        input_dim: int.
        timesteps: int, input timestep dimension.
        batch_size: int.
        intermediate_dim: int, output shape of LSTM. 
        latent_dim: int, latent z-layer shape. 
        epsilon_std: float, z-layer sigma.
    # References
        - [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
        - [Generating sentences from a continuous space](https://arxiv.org/abs/1511.06349)
    """
    x = Input(shape=(timesteps, input_dim,))

    # LSTM encoding
    h = LSTM(intermediate_dim)(x)

    # VAE Z layer
    z_mean = Dense(latent_dim)(h)
    z_log_sigma = Dense(latent_dim)(h)
    
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim),
                                  mean=0., stddev=epsilon_std)
        return z_mean + z_log_sigma * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    # so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
    
    # decoded LSTM layer
    decoder_h = LSTM(intermediate_dim, return_sequences=True)
    decoder_mean = LSTM(input_dim, return_sequences=True)

    h_decoded = RepeatVector(timesteps)(z)
    h_decoded = decoder_h(h_decoded)

    # decoded layer
    x_decoded_mean = decoder_mean(h_decoded)
    
    # end-to-end autoencoder
    vae = Model(x, x_decoded_mean)

    # encoder, from inputs to latent space
    encoder = Model(x, z_mean)

    # generator, from latent space to reconstructed inputs
    decoder_input = Input(shape=(latent_dim,))

    _h_decoded = RepeatVector(timesteps)(decoder_input)
    _h_decoded = decoder_h(_h_decoded)

    _x_decoded_mean = decoder_mean(_h_decoded)
    generator = Model(decoder_input, _x_decoded_mean)
    
    def vae_loss(x, x_decoded_mean):
        xent_loss = mse(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    vae.compile(optimizer='rmsprop', loss=vae_loss)
    
    return vae, encoder, generator

vae, enc, gen = create_lstm_vae(max_len, 
        timesteps=3, 
        batch_size=batch_size, 
        intermediate_dim=32,
        latent_dim=100,
        epsilon_std=1.)

vae.fit(x_partial_train, x_partial_train, epochs=20)

ValueError: Error when checking input: expected input_8 to have 3 dimensions, but got array with shape (11985, 20)

In [29]:
len(x_partial_train)

11985

In [30]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5" 
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint('models', 'vae_seq2seq_test_very_high_std')



vae.fit(x_partial_train, x_partial_train,
     shuffle=True,
     epochs=100,
     batch_size=batch_size,
     validation_data=(x_val, x_val), callbacks=[checkpointer])

#print(K.eval(vae.optimizer.lr))
#K.set_value(vae.optimizer.lr, 0.01)

vae.save('models/vae_lstm.h5')
#vae.load_weights('models/vae_seq2seq_test.h5')

Train on 11985 samples, validate on 1500 samples
Epoch 1/100
11900/11985 [============================>.] - ETA: 0s - loss: nan - kl_loss: nan

2022-07-26 13:01:16.869252: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: 2 root error(s) found.
  (0) INVALID_ARGUMENT: required broadcastable shapes
	 [[{{node lambda_2/mul}}]]
	 [[metrics_2/kl_loss/Identity/_473]]
  (1) INVALID_ARGUMENT: required broadcastable shapes
	 [[{{node lambda_2/mul}}]]
0 successful operations.
0 derived errors ignored.

In [29]:
# build a model to project sentences on the latent space
encoder = Model(x, z_mean)

# build a generator that can sample sentences from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(repeated_context(decoder_input))
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
generator = Model(decoder_input, _x_decoded_mean)

In [33]:
index2word = {v: k for k, v in word_index.items()}
sent_encoded = encoder.predict(x_val, batch_size = 16)
x_test_reconstructed = generator.predict(sent_encoded)
                                         
sent_idx = 672
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[sent_idx])
#np.apply_along_axis(np.max, 1, x_test_reconstructed[sent_idx])
#np.max(np.apply_along_axis(np.max, 1, x_test_reconstructed[sent_idx]))
word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
word_list
original_sent = list(np.vectorize(index2word.get)(x_val[sent_idx]))
original_sent

ValueError: in user code:

    File "/home/paul/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/home/paul/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/paul/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/home/paul/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/home/paul/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/paul/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 200, in assert_input_compatibility
        raise ValueErr